In [47]:
!pip install pandas-plink limix-lmm

In [48]:
import numpy as np
import pdb 
from pandas_plink import read_plink 
from limix_lmm import LMM
import pylab as plt
import os
import pandas as pd

In [49]:
import scipy
import scipy.stats as st
scipy.dot = np.dot
scipy.log = np.log
scipy.sqrt = np.sqrt
scipy.sign = np.sign

In [50]:
!wget -O ALL.chr22_GRCh38.genotypes.20170504.zip "https://www.dropbox.com/s/86fdj3i7ir3vap7/ALL.chr22_GRCh38.genotypes.20170504.zip?dl=1"

--2026-02-09 17:31:08--  https://www.dropbox.com/s/86fdj3i7ir3vap7/ALL.chr22_GRCh38.genotypes.20170504.zip?dl=1
Auflösen des Hostnamens www.dropbox.com (www.dropbox.com)… 2620:100:6022:18::a27d:4212, 162.125.66.18
Verbindungsaufbau zu www.dropbox.com (www.dropbox.com)|2620:100:6022:18::a27d:4212|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 302 Found
Platz: https://www.dropbox.com/scl/fi/2j079oew31gwd2hzajlil/ALL.chr22_GRCh38.genotypes.20170504.zip?rlkey=7307x4sjq9lkvhwu3aivh5uut&dl=1 [folgend]
--2026-02-09 17:31:08--  https://www.dropbox.com/scl/fi/2j079oew31gwd2hzajlil/ALL.chr22_GRCh38.genotypes.20170504.zip?rlkey=7307x4sjq9lkvhwu3aivh5uut&dl=1
Wiederverwendung der bestehenden Verbindung zu [www.dropbox.com]:443.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 302 Found
Platz: https://uc565dff6f48845176809d899b17.dl.dropboxusercontent.com/cd/0/inline/C6knb9yDn2nuIiEKHKvYuX2uKu1JQq57Z9O3fhkRqb4TL2fC63AC2wyl_PGwByfkjjxr_RQGvUL0_0m6xDNaTzymz8DTyf3vJKjd4

In [51]:
!unzip -o -q ALL.chr22_GRCh38.genotypes.20170504.zip && rm ALL.chr22_GRCh38.genotypes.20170504.zip

In [67]:
bfile = 'ALL.chr22_GRCh38.genotypes.20170504' 
bim, fam, G = read_plink(bfile)

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 13.29it/s]


In [ ]:
print(len(bim))

G = G.compute()

snp_missing = np.mean(np.isnan(G), axis=1)  
keep_missing = snp_missing < 0.01           # missing rate > 0.01 (0.02, or 0.05) -> remove SNPs, that are missing in more than 1% of individuals

G = G[keep_missing, :]
bim = bim.loc[keep_missing].copy()

bim.reset_index(drop=True, inplace=True)
bim['i'] = np.arange(len(bim))

allele_freq = np.nanmean(G, axis=1) / 2.0     # Calculate allele frequency (values are 0, 1, 2 -> divide by 2)
maf = np.minimum(allele_freq, 1 - allele_freq)

keep_maf = maf > 0.01  # maf < 0.01 

keep_unique = ~bim['snp'].duplicated()     # removing duplicated 

filtered = keep_maf & keep_unique.values

G = G[filtered, :]
bim = bim.loc[filtered].copy()

bim.reset_index(drop=True, inplace=True)
bim['i'] = np.arange(len(bim))

print(len(bim))

109827
109788


In [10]:
def extract_region(bim, G, chrom, start, end): 
    I1 = bim['chrom']==str(chrom)
    I2 = bim['pos'] > start
    I3 = bim['pos'] < end
    Ikeep = I1 * I2 * I3
    bim = bim.loc[Ikeep].copy()
    G = G[bim['i'].values] 
    bim['i'] = np.arange(len(bim)) 
    return bim, G

In [15]:
def subset_individuals(fam, G, keep_idxs): 
    fam = fam.iloc[keep_idxs]
    G = G[:, keep_idxs]
    return fam, G

In [16]:
N = 1000
np.random.seed(0)
idxs = np.sort(np.random.permutation(fam.shape[0])[:N]) 
fams, Gs = subset_individuals(fam, G, idxs)

In [17]:
chrom = 22
start = 30797531
size = 100000
bimr, Gr = extract_region(bim, Gs, chrom, start, start + size)

In [ ]:
def simulate_pheno(X, idx_caus, var_expl, direction=None):
# Ensure that the number of causal variant indices matches the number of variances explained. assert len(idx_caus) == len(var_expl)
    # If no direction is provided, randomly assign a positive or negative direction for each causal variant.
    if direction is None:
        direction = 2. * (np.random.rand(len(idx_caus)) > 0.5) - 1.
        # Ensure that the number of directions matches the number of causal variant indices.
    assert len(idx_caus) == len(direction)
        # Compute the remaining variance after accounting for the variance explained by the causal variants.
    ve = 1 - var_expl.sum()
    # Ensure that the total variance explained by causal variants is less than 1. assert ve > 0, 'sum(var_expl) should be < 1'
        # Compute the effect sizes for the causal variants based on the variance they explain and their direction.
    beta = np.sqrt(var_expl) * direction
        # Extract the columns of X corresponding to the causal variants and standardize them.
    Xc = X[:, idx_caus]
    Xc = (Xc - Xc.mean(0)) / Xc.std(0)
        # Compute the genetic component of the phenotype.
    yg = Xc.dot(beta)[:, None]
    # Compute the noise component of the phenotype. yn = np.sqrt(ve) * np.random.randn(X.shape[0], 1)
        # Sum the genetic and noise components to get the simulated phenotype.
    y = yg + yn
        # Initialize the real effect sizes for all variants in X as zeros.
    beta_real = np.zeros(X.shape[1])
    # Update the real effect sizes for the causal variants. beta_real[idx_caus] = beta
        # Standardize the phenotypic values to have mean 0 and standard deviation 1.
    ystd = y.std()
    y = (y - y.mean()) / ystd
    # Adjust the real effect sizes accordingly after standardizing y. beta_real = beta_real / ystd
    return y, beta_real